# Multi-Echo Denoising with `tedana`

In this analysis tutorial, we will use `tedana` {cite:p}`DuPre2021` to perform multi-echo denoising.

Specifically, we will use {py:func}`tedana.workflows.tedana_workflow`.

In [1]:
import json
import os
from glob import glob
from pprint import pprint

import pandas as pd
from IPython.display import HTML, display
from repo2data.repo2data import Repo2Data
from tedana import workflows

# Install the data if running locally, or point to cached data if running on neurolibre
DATA_REQ_FILE = os.path.join("../binder/data_requirement.json")

# Download data
repo2data = Repo2Data(DATA_REQ_FILE)
data_path = repo2data.install()
data_path = os.path.abspath(data_path[0])

---- repo2data starting ----
/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/site-packages/repo2data
Config from file :
../binder/data_requirement.json
Destination:
./../data/multi-echo-data-analysis

Info : ./../data/multi-echo-data-analysis already downloaded


/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
func_dir = os.path.join(data_path, "func/")
data_files = [
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-1_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-2_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-3_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
    os.path.join(
        func_dir,
        "sub-04570_task-rest_echo-4_space-scanner_desc-partialPreproc_bold.nii.gz",
    ),
]
echo_times = [12.0, 28.0, 44.0, 60.0]
mask_file = os.path.join(
    func_dir, "sub-04570_task-rest_space-scanner_desc-brain_mask.nii.gz"
)
confounds_file = os.path.join(
    func_dir, "sub-04570_task-rest_desc-confounds_timeseries.tsv"
)

out_dir = os.path.join(data_path, "tedana")

In [3]:
workflows.tedana_workflow(
    data_files,
    echo_times,
    out_dir=out_dir,
    mask=mask_file,
    prefix="sub-04570_task-rest_space-scanner",
    fittype="curvefit",
    tedpca="mdl",
    verbose=True,
    gscontrol=["mir"],
)

INFO     tedana:tedana_workflow:569 Using output directory: /home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/tedana


INFO     tedana:tedana_workflow:587 Initializing and validating component selection tree


WARNING  component_selector:validate_tree:141 Decision tree includes fields that are not used or logged ['_comment']


INFO     component_selector:__init__:326 Performing component selection with tedana_orig_decision_tree


INFO     component_selector:__init__:327 Very similar to the decision tree designed by Prantik Kundu


...

/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/site-packages/tedana/io.py:833: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  nii = new_img_like(ref_img, newdata, affine=affine, copy_header=copy_header)


/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/site-packages/tedana/io.py:833: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  nii = new_img_like(ref_img, newdata, affine=affine, copy_header=copy_header)


/opt/hostedtoolcache/Python/3.10.15/x64/lib/python3.10/site-packages/nilearn/plotting/img_plotting.py:1317: UserWarning: Non-finite values detected. These values will be replaced with zeros.
  safe_get_data(stat_map_img, ensure_finite=True),


INFO     tedana:tedana_workflow:1058 Generating dynamic report


INFO     tedana:tedana_workflow:1061 Workflow completed


The tedana workflow writes out a number of files.

In [4]:
out_files = sorted(glob(os.path.join(out_dir, "*")))
out_files = [os.path.basename(f) for f in out_files]
print("\n".join(out_files))

figures
sub-04570_task-rest_space-scanner_S0map.nii.gz
sub-04570_task-rest_space-scanner_T2starmap.nii.gz
sub-04570_task-rest_space-scanner_dataset_description.json
sub-04570_task-rest_space-scanner_desc-ICAAcceptedMIRDenoised_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAccepted_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAccepted_stat-z_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAAveragingWeights_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICACrossComponent_metrics.json
sub-04570_task-rest_space-scanner_desc-ICAMIRDenoised_mixing.tsv
sub-04570_task-rest_space-scanner_desc-ICAS0_stat-F_statmap.nii.gz
sub-04570_task-rest_space-scanner_desc-ICAT2_stat-F_statmap.nii.gz
sub-04570_task-rest_space-scanner_desc-ICA_components.nii.gz
sub-04570_task-rest_space-scanner_desc-ICA_decision_tree.json
sub-04570_task-rest_space-scanner_desc-ICA_decomposition.json
sub-04570_task-rest_space-scanner_desc-ICA_mixing.tsv
sub-04570_task-rest_space

In [5]:
metrics = pd.read_table(
    os.path.join(out_dir, "sub-04570_task-rest_space-scanner_desc-tedana_metrics.tsv")
)

In [6]:
def color_rejected_red(series):
    """Color rejected components red."""
    return [
        "color: red" if series["classification"] == "rejected" else "" for v in series
    ]


metrics.style.apply(color_rejected_red, axis=1)

,Component,kappa,rho,variance explained,normalized variance explained,countsigFT2,countsigFS0,dice_FT2,dice_FS0,countnoise,signal-noise_t,signal-noise_p,d_table_score,optimal sign,varex kappa ratio,d_table_score_node20,classification,classification_tags
0,ICA_00,23.963910,18.845160,0.974387,0.014406,813,766,0.000000,0.000000,1223,-2.264125,0.025652,31.200000,1,3.124711,nan,rejected,Unlikely BOLD
1,ICA_01,18.386390,15.105416,0.209236,0.002638,322,163,0.243200,0.000000,1228,0.000000,0.000000,31.200000,1,0.874533,nan,accepted,Low variance
2,ICA_02,53.904796,11.570936,0.365079,0.004694,1849,375,0.621872,0.252033,880,10.298052,0.000000,7.600000,-1,0.520470,6.600000,accepted,Likely BOLD
3,ICA_03,22.443964,29.659039,0.394177,0.005023,840,1175,0.336829,0.364685,1183,0.000000,0.000000,25.800000,1,1.349670,nan,rejected,Unlikely BOLD
4,ICA_04,36.402447,36.417731,1.079447,0.010505,2030,1616,0.447545,0.424479,1136,7.114071,0.000000,14.400000,-1,2.278803,nan,rejected,Unlikely BOLD
5,ICA_05,14.776330,53.185372,0.539564,0.012173,280,786,0.000000,0.362080,509,-8.452758,0.000000,31.000000,-1,2.806160,nan,rejected,Unlikely BOLD
6,ICA_06,35.682686,41.152975,1.411969,0.016422,722,729,0.268085,0.332160,1050,6.150607,0.000002,19.000000,-1,3.040910,nan,rejected,Unlikely BOLD
7,ICA_07,102.876143,10.929019,0.754819,0.009029,3277,362,0.669988,0.166436,613,6.615788,0.000000,2.800000,-1,0.563851,2.400000,accepted,Likely BOLD
8,ICA_08,14.768110,20.934533,0.273712,0.003667,226,300,0.059908,0.384615,997,-6.963965,0.000000,34.200000,-1,1.424309,nan,rejected,Unlikely BOLD
9,ICA_09,21.605372,17.761140,0.307787,0.004137,388,282,0.000000,0.000000,1225,0.000000,0.000000,32.000000,1,1.094775,nan,accepted,Low variance


In [7]:
with open(
    os.path.join(out_dir, "sub-04570_task-rest_space-scanner_desc-tedana_metrics.json"),
    "r",
) as fo:
    data = json.load(fo)

first_five_keys = list(data.keys())[:5]
reduced_data = {k: data[k] for k in first_five_keys}
pprint(reduced_data)

{'Component': {'Description': 'The unique identifier of each component. This '
                              'identifier matches column names in the mixing '
                              'matrix TSV file.',
               'LongName': 'Component identifier'},
 'classification_tags': {'Description': 'A single tag or a comma separated '
                                        'list of tags to describe why a '
                                        'component received its classification',
                         'LongName': 'Component classification tags'},
 'countnoise': {'Description': "Number of 'noise' voxels (voxels highly "
                               'weighted for component, but not from clusters) '
                               'from each component.',
                'LongName': 'Noise voxel count',
                'Units': 'voxel'},
 'countsigFS0': {'Description': 'Number of significant voxels from the '
                                'cluster-extent thresholded S0 model '

In [8]:
df = pd.DataFrame.from_dict(data, orient="index")
df = df.fillna("n/a")
display(HTML(df.to_html()))

,Description,LongName,Units,Levels
Component,The unique identifier of each component. This identifier matches column names in the mixing matrix TSV file.,Component identifier,n/a,n/a
classification_tags,A single tag or a comma separated list of tags to describe why a component received its classification,Component classification tags,n/a,n/a
countnoise,"Number of 'noise' voxels (voxels highly weighted for component, but not from clusters) from each component.",Noise voxel count,voxel,n/a
countsigFS0,Number of significant voxels from the cluster-extent thresholded S0 model F-statistic map for each component.,S0 model F-statistic map significant voxel count,voxel,n/a
countsigFT2,Number of significant voxels from the cluster-extent thresholded T2 model F-statistic map for each component.,T2 model F-statistic map significant voxel count,voxel,n/a
d_table_score,"Summary score compiled from five metrics, with smaller values (i.e., higher ranks) indicating more BOLD dependence and less noise.",Decision table score,arbitrary,n/a
dice_FS0,Dice value of cluster-extent thresholded maps of S0-model betas and F-statistics.,S0 model beta map-F-statistic map Dice similarity index,arbitrary,n/a
dice_FT2,Dice value of cluster-extent thresholded maps of T2-model betas and F-statistics.,T2 model beta map-F-statistic map Dice similarity index,arbitrary,n/a
kappa,"A pseudo-F-statistic indicating TE-dependence of the component. This metric is calculated by computing fit to the TE-dependence model at each voxel, and then performing a weighted average based on the voxel-wise weights of the component.",Kappa,arbitrary,n/a
normalized variance explained,Normalized variance explained in the optimally combined data of each component.On a scale from 0 to 1.,Normalized variance explained,arbitrary,n/a


In [9]:
report = os.path.join(out_dir, "tedana_report.html")
with open(report, "r") as fo:
    report_data = fo.read()

figures_dir = os.path.relpath(os.path.join(out_dir, "figures"), os.getcwd())
report_data = report_data.replace("./figures", figures_dir)

display(HTML(report_data))

FileNotFoundError: [Errno 2] No such file or directory: '/home/runner/work/multi-echo-data-analysis/multi-echo-data-analysis/data/multi-echo-data-analysis/tedana/tedana_report.html'